In [6]:
#KNN
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pylab as plt
from pathlib import Path

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart

In [2]:
China_df = pd.read_csv('model.csv')
China_df.head()

Unnamed: 0  days_before_qua  exsisting_confirmed_on_qua  \
0    beijing                6                          49   
1    tianjin                5                          10   
2      hebei                4                          12   
3     shanxi                4                           9   
4  neimenggu                3                           5   

   existing_confirmed_TP existing_confirmed_TP vs total_pop  \
0                    295                              0.01%   
1                     99                              0.01%   
2                    211                              0.00%   
3                     96                              0.00%   
4                     66                              0.00%   

  Infetion Rate above 0.01%_TP(Y/N)  accumulated_confirmed_TP  \
0                                 Y                       366   
1                                 N                       112   
2                                 N                       283   
3                                 N                       122   
4                                 N                        75   

   accumulated_death_TP  accumulated_recovered_TP  total_pop  ...  \
0                     3                        68    2155244  ...   
1                     2                        11    1560244  ...   
2                     3                        69    7549635  ...   
3                     0                        26    3717196  ...   
4                     0                         9    2534269  ...   

   total_passenger_volume  area_passenger_volume  actual_passenger_volume  \
0             715093.2625              43.683156              4305153.315   
1             153157.5500              13.196411              1516259.745   
2             185798.0225               0.993333              3266948.562   
3             130117.6525               0.832055              1387921.627   
4             102200.8300               0.089246              1349050.956   

   Unnamed: 19  air_passenger_volume  railway_passenger_volume  \
0  4305153.315               10750.0                  384842.0   
1          NaN                2359.0                   40834.0   
2          NaN                1391.0                    8760.0   
3          NaN                1843.0                       0.0   
4          NaN                2530.0                       0.0   

   bus_passenger_volume  hospitals  hospital_beds  quarantine_level  
0              318976.0        219         116400          0.166102  
1              109725.0        136          60300          0.101010  
2              175242.0        662         320700          0.056872  
3              128046.0        181         163600          0.093750  
4               99378.0        243         126400          0.075758  

[5 rows x 26 columns]

In [3]:
China_df1 = China_df[['pop_density','avg_age','hospital_beds','quarantine_level','days_before_qua','Infetion Rate above 0.01%_TP(Y/N)']]

In [6]:
China_df1.head()

pop_density    avg_age  hospital_beds  quarantine_level  days_before_qua  \
0   131.658155  39.755631         116400          0.166102                6   
1   134.434258  39.710867          60300          0.101010                5   
2    40.362667  37.616780         320700          0.056872                4   
3    23.770126  37.786333         163600          0.093750                4   
4     2.213034  38.388404         126400          0.075758                3   

  Infetion Rate above 0.01%_TP(Y/N)  
0                                 Y  
1                                 N  
2                                 N  
3                                 N  
4                                 N

In [20]:
trainData, validData = train_test_split(China_df1, test_size=0.4, random_state=26)
print(trainData.shape, validData.shape)
newregion = pd.DataFrame([{'pop_density': 839,'avg_age': 40,'hospital_beds': 15000,'quarantine_level': 0.05,'days_before_qua':105}])
newregion

(20, 6) (14, 6)


pop_density  avg_age  hospital_beds  quarantine_level  days_before_qua
0          839       40          15000              0.05              105

In [21]:
scaler = preprocessing.StandardScaler()
scaler.fit(trainData[['pop_density','avg_age','hospital_beds','quarantine_level','days_before_qua']])

# Transform the full dataset
ChinaNorm = pd.concat([pd.DataFrame(scaler.transform(China_df1[['pop_density','avg_age','hospital_beds','quarantine_level','days_before_qua']]), 
                                    columns=['zpop_density','zavg_age','zhospital_beds','zquarantine_level','zdays_before_qua']),
                      China_df1[['Infetion Rate above 0.01%_TP(Y/N)']]], axis=1)
trainNorm = ChinaNorm.iloc[trainData.index]
validNorm = ChinaNorm.iloc[validData.index]
newregionNorm = pd.DataFrame(scaler.transform(newregion), columns=['zpop_density','zavg_age','zhospital_beds','zquarantine_level','zdays_before_qua'])

In [22]:
knn = NearestNeighbors(n_neighbors=7)
knn.fit(trainNorm[['zpop_density','zavg_age','zhospital_beds','zquarantine_level','zdays_before_qua']])
distances, indices = knn.kneighbors(newregionNorm)
print(trainNorm.iloc[indices[0], :])

    zpop_density  zavg_age  zhospital_beds  zquarantine_level  \
33      0.168083  1.073399       -0.302997           2.739746   
31      4.339479  1.756257       -1.229999           0.783866   
1      -0.185531  0.895327       -1.081671          -0.171556   
6      -0.267560  0.793361       -0.483610          -1.014694   
4      -0.275731  0.164279       -0.588140          -0.639966   
7      -0.271520  1.076590        0.033067          -1.339526   
0      -0.187425  0.920072       -0.662804           1.035830   

    zdays_before_qua Infetion Rate above 0.01%_TP(Y/N)  
33          2.993066                                 N  
31          2.993066                                 N  
1          -0.260934                                 N  
6          -0.383726                                 N  
4          -0.383726                                 N  
7          -0.383726                                 N  
0          -0.199538                                 Y  


In [18]:
train_X = trainNorm[['zpop_density','zavg_age','zhospital_beds','zquarantine_level','zdays_before_qua']]
train_y = trainNorm['Infetion Rate above 0.01%_TP(Y/N)']
valid_X = validNorm[['zpop_density','zavg_age','zhospital_beds','zquarantine_level','zdays_before_qua']]
valid_y = validNorm['Infetion Rate above 0.01%_TP(Y/N)']

# Train a classifier for different values of k
results = []
for k in range(1, 10):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

   k  accuracy
0  1  0.642857
1  2  0.714286
2  3  0.714286
3  4  0.714286
4  5  0.714286
5  6  0.785714
6  7  0.785714
7  8  0.785714
8  9  0.714286


In [11]:
#Naive Bayes

predictors = ['pop_density','avg_age','hospital_beds','quarantine_level','days_before_qua']
outcome = 'Infetion Rate above 0.01%_TP(Y/N)'

X = pd.get_dummies(China_df1[predictors])
y = (China_df1[outcome] == 'Y').astype(int)
classes = ['Y', 'N']

# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

# run naive Bayes
China_nb = MultinomialNB(alpha=0.01)
China_nb.fit(X_train, y_train)

# predict probabilities
predProb_train = China_nb.predict_proba(X_train)
predProb_valid = China_nb.predict_proba(X_valid)

# predict class
y_valid_pred = China_nb.predict(X_valid)
y_train_pred = China_nb.predict(X_train)


In [9]:
classificationSummary(y_train, y_train_pred, class_names=classes) 

print()

classificationSummary(y_valid, y_valid_pred, class_names=classes) 

Confusion Matrix (Accuracy 0.4500)

       Prediction
Actual  Y  N
     Y  3 10
     N  1  6

Confusion Matrix (Accuracy 0.2143)

       Prediction
Actual  Y  N
     Y  1 11
     N  0  2
